In [1]:
import time
notebook_start_time = time.time()

# Download and Inspect on the Dataset

In [2]:
import urllib.request

url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/"
        "refs/heads/main/ch02/01_main-chapter-code/"
        "the-verdict.txt")
file_path = "../datasets/the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('../datasets/the-verdict.txt', <http.client.HTTPMessage at 0x24a89d9c7a0>)

In [3]:
with open("../datasets/the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

print("total number of characters:", len(raw_text))
print(raw_text[:500])

total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)

"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it'


## Split texts into words

In [4]:
import re

experiment_text = "Hello, World! Let's test this, --sentence."

# the splitting delimiters included in the output
splitted_word_list = re.split(r'([,.:;?_!"()\'\s]|--)', experiment_text)
print(splitted_word_list)


# not included
# splitted_word_list2 = re.split(r'[,.]|\s', experiment_text)
# print(splitted_word_list2)

['Hello', ',', '', ' ', 'World', '!', '', ' ', 'Let', "'", 's', ' ', 'test', ' ', 'this', ',', '', ' ', '', '--', 'sentence', '.', '']


In [5]:
# get rid of white-space
splitted_word_list = [word for word in splitted_word_list if word.strip()]
print(splitted_word_list)

['Hello', ',', 'World', '!', 'Let', "'", 's', 'test', 'this', ',', '--', 'sentence', '.']


Let's apply it to the text

In [6]:
preprocessed = re.split(r'([,.:;?_!"()\'\s]|--)', raw_text)
preprocessed = [word for word in preprocessed if word.strip()]
print(preprocessed[:50])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself']


## Convert tokens into token IDs

In [7]:
set_of_all_words = sorted(set(preprocessed))
print(set_of_all_words)

['!', '"', "'", '(', ')', ',', '--', '.', ':', ';', '?', 'A', 'Ah', 'Among', 'And', 'Are', 'Arrt', 'As', 'At', 'Be', 'Begin', 'Burlington', 'But', 'By', 'Carlo', 'Chicago', 'Claude', 'Come', 'Croft', 'Destroyed', 'Devonshire', 'Don', 'Dubarry', 'Emperors', 'Florence', 'For', 'Gallery', 'Gideon', 'Gisburn', 'Gisburns', 'Grafton', 'Greek', 'Grindle', 'Grindles', 'HAD', 'Had', 'Hang', 'Has', 'He', 'Her', 'Hermia', 'His', 'How', 'I', 'If', 'In', 'It', 'Jack', 'Jove', 'Just', 'Lord', 'Made', 'Miss', 'Money', 'Monte', 'Moon-dancers', 'Mr', 'Mrs', 'My', 'Never', 'No', 'Now', 'Nutley', 'Of', 'Oh', 'On', 'Once', 'Only', 'Or', 'Perhaps', 'Poor', 'Professional', 'Renaissance', 'Rickham', 'Riviera', 'Rome', 'Russian', 'Sevres', 'She', 'Stroud', 'Strouds', 'Suddenly', 'That', 'The', 'Then', 'There', 'They', 'This', 'Those', 'Though', 'Thwing', 'Thwings', 'To', 'Usually', 'Venetian', 'Victor', 'Was', 'We', 'Well', 'What', 'When', 'Why', 'Yes', 'You', '_', 'a', 'abdication', 'able', 'about', 'above',

In [8]:
vocab_size = len(set_of_all_words)
print(vocab_size)

1130


In [9]:
# print some vocab
vocab = {token:integer for integer, token in enumerate(set_of_all_words)}

for i, token in enumerate(vocab.items()):
  print(token)
  if i == 50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


## Implement a Complete Simple Tokenizer

In [10]:
class SimpleTokenizerV1:
  def __init__(self, vocab):
    self.word_to_id = vocab
    self.id_to_word = {id:word for word, id in vocab.items()}

  # word --> id (tokenize)
  def encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\'\s]|--)', text)
    preprocessed = [word for word in preprocessed if word.strip()]
    id_list = [self.word_to_id[word] for word in preprocessed]
    return id_list

  # id --> word (detokenize)
  def decode(self, id_list):
    word_list = [self.id_to_word[id] for id in id_list]
    text_with_white_space = " ".join(word_list)
    # remove space before punctuations
    text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1', text_with_white_space)
    return text

In [11]:
simple_tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
Mrs. Gisburn said with pardonable pride."""

id_list = simple_tokenizer.encode(text)
print(id_list)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [12]:
print(simple_tokenizer.decode(id_list))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


Our simple tokenizer works, but it cannot work on the data out of the text.

## Adding Special Context Tokens

In [13]:
list_of_all_words = sorted(list(set(preprocessed)))
list_of_all_words.extend(["<|unk|>", "<|endoftext|>"])
vocab = {word:id for id, word in enumerate(list_of_all_words)}

print(len(vocab.items()))

for i, word in enumerate(list(vocab.items())[-10:]):
  print(word)

1132
('year', 1122)
('years', 1123)
('yellow', 1124)
('yet', 1125)
('you', 1126)
('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|unk|>', 1130)
('<|endoftext|>', 1131)


## Implement a Simple Tokenizer That Handles Unknown Words

In [14]:
class SimpleTokenizerV2:
  def __init__(self, vocab):
    self.word_to_id = vocab
    self.id_to_word = {id:word for word, id in vocab.items()}

  # word --> id (tokenizer)
  def encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\'\s]|--)', text)
    preprocessed = [word for word in preprocessed if word.strip()]
    # handle unknown words
    word_list = [word if word in self.word_to_id
              else "<|unk|>" for word in preprocessed]
    id_list = [self.word_to_id[word] for word in word_list]
    return id_list

  # id --> word (detokenizer)
  def decode(self, id_list):
    word_list = [self.id_to_word[id] for id in id_list]
    text_with_white_space = " ".join(word_list)
    # remove space before punctuations
    text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1', text_with_white_space)
    return text

Let's test our new tokenizer:

In [15]:
text1 = """"In the midst of winter, I found, within me,
 an invincible summer" said Albert Camus"""
text2 = """Happy Hunger Games, and may the odds be ever in your favor."""

text = " <|endoftext|> ".join((text1, text2))
print(text)

"In the midst of winter, I found, within me,
 an invincible summer" said Albert Camus <|endoftext|> Happy Hunger Games, and may the odds be ever in your favor.


In [16]:
tokenizerv2 = SimpleTokenizerV2(vocab)
print(tokenizerv2.encode(text))

[1, 55, 988, 1130, 722, 1130, 5, 53, 469, 5, 1130, 663, 5, 156, 1130, 1130, 1, 851, 1130, 1130, 1131, 1130, 1130, 1130, 5, 157, 662, 988, 1130, 198, 401, 568, 1128, 1130, 7]


In [17]:
print(tokenizerv2.decode(tokenizerv2.encode(text)))

" In the <|unk|> of <|unk|>, I found, <|unk|> me, an <|unk|> <|unk|>" said <|unk|> <|unk|> <|endoftext|> <|unk|> <|unk|> <|unk|>, and may the <|unk|> be ever in your <|unk|>.


## Byte Pair Encoding (PBE)

To handle unknown words at ease, we tokenize subword instead:

[tiktoken doc](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb)

In [18]:
from importlib.metadata import version
import tiktoken

print("tiktoken version: ", version("tiktoken"))
print(tiktoken.encoding_for_model("gpt-3.5-turbo"))
print(tiktoken.encoding_for_model("gpt-3.5-turbo").n_vocab)
print(tiktoken.list_encoding_names())

tiktoken version:  0.9.0
<Encoding 'cl100k_base'>
100277
['gpt2', 'r50k_base', 'p50k_base', 'p50k_edit', 'cl100k_base', 'o200k_base']


In [19]:
bpe_tokenizer = tiktoken.get_encoding("gpt2")
print(bpe_tokenizer.n_vocab)
print(text)

50257
"In the midst of winter, I found, within me,
 an invincible summer" said Albert Camus <|endoftext|> Happy Hunger Games, and may the odds be ever in your favor.


In [20]:
id_list = bpe_tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(id_list)

[1, 818, 262, 15925, 286, 7374, 11, 314, 1043, 11, 1626, 502, 11, 198, 281, 46038, 3931, 1, 531, 9966, 7298, 385, 220, 50256, 14628, 32367, 5776, 11, 290, 743, 262, 10402, 307, 1683, 287, 534, 2661, 13]


In [21]:
decoded_text = bpe_tokenizer.decode(id_list)
print(decoded_text)

"In the midst of winter, I found, within me,
 an invincible summer" said Albert Camus <|endoftext|> Happy Hunger Games, and may the odds be ever in your favor.


In [22]:
# let's decode into byte
decoded_text_byte = [bpe_tokenizer.decode_single_token_bytes(token)
                    for token in id_list]
for token in decoded_text_byte:
  #print(token.decode("utf-8"))
  print(token)

b'"'
b'In'
b' the'
b' midst'
b' of'
b' winter'
b','
b' I'
b' found'
b','
b' within'
b' me'
b','
b'\n'
b' an'
b' invincible'
b' summer'
b'"'
b' said'
b' Albert'
b' Cam'
b'us'
b' '
b'<|endoftext|>'
b' Happy'
b' Hunger'
b' Games'
b','
b' and'
b' may'
b' the'
b' odds'
b' be'
b' ever'
b' in'
b' your'
b' favor'
b'.'


## Comparing Trained LLM Tokenizers From HuggingFace

[color combination](https://coolors.co/)

In [23]:
# recommended
# colors_list = [ #'R;B;G'
#     '102;194;165', '252;141;98', '141;160;203',
#     '231;138;195', '166;216;84', '255;217;47'
# ]


# colors_list = [ #'R;B;G'
#     '244;241;222', '224;122;95', '61;64;91',
#     '129;178;154', '242;204;143',
# ]


# colors_list = [ #'R;B;G'
#     '255;190;11', '251;86;7', '255;0;110',
#     '131;56;236', '58;134;255',
# ]


# # recommended
# colors_list = [ #'R;B;G'
#     '251;248;204', '253;228;207', '255;207;210',
#     '241;192;232', '207;186;240', '163;196;243',
#     '144;219;244', '142;236;245', '152;245;225',
#     '185;251;192',
# ]

# recommended
colors_list = [ #'R;B;G'
    '255;173;173', '255;214;165', '253;255;182',
    '253;255;182', '202;255;191', '155;246;255',
    '160;196;255', '189;178;255', '255;198;225',
    '255;255;252',
]

In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

In [25]:
text = """
English and CAPITALIZATION
🎵 大模型語言
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600

def greet():
  while True:
    print("Hello World!")
"""

In [26]:
show_tokens(text, "bert-base-uncased")

[CLS] english and capital ##ization [UNK] 大 [UNK] [UNK] 語 [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 def greet ( ) : while true : print ( " hello world ! " ) [SEP] 

In [27]:
show_tokens(text, "bert-base-cased")

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] 大 [UNK] [UNK] [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 def greet ( ) : while True : print ( " Hello World ! " ) [SEP] 

In [28]:
show_tokens(text, "gpt2")


 English  and  CAP ITAL IZ ATION 
 � � �  � � � � � � � � � � � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 
 def  greet (): 
    while  True : 
        print (" Hello  World !" ) 
 

In [29]:
show_tokens(text, "google/flan-t5-small")

English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600 de f greet () : while True : print ( " H ello World !" )  </s> 

In [30]:
# The official is `tiktoken` but this the same tokenizer on the HF platform
show_tokens(text, "Xenova/gpt-4")


 English  and  CAPITAL IZATION 
 � � �  � � 模 型 � � 言 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 12 . 0 * 50 = 600 

 def  greet ():
    while  True :
      print (" Hello  World !")
 

In [31]:
# You need to request access before being able to use this tokenizer
show_tokens(text, "bigcode/starcoder2-15b")


 English  and  CAPITAL IZATION 
 � � �  大 模型 語 言 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 
 def  g reet (): 
   while  True : 
     print (" Hello  World !") 
 

In [32]:
show_tokens(text, "facebook/galactica-1.3b")


 English  and  CAP ITAL IZATION 
 � � � �  � � � � � � � � � � � � � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : "      "  Three  t abs :   "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 
 def  gre et ( ) : 
    while  True : 
      print ( " H ello  World ! " ) 
 

In [33]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

 
 English and C AP IT AL IZ ATION 
 � � � �  大 模 型 語 言 
 show _ to kens False None elif == >= else : two tabs :"    " Three tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 
 def gre et (): 
  while True : 
    print (" Hello World ! ") 
 

# Dataset and DataLoader

In [34]:
encoded_text = bpe_tokenizer.encode(raw_text)
print(len(encoded_text))
print(len(raw_text))

5145
20479


In [35]:
encoded_sample = encoded_text[:100]
print(encoded_sample)

[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257, 7026, 15632, 438, 2016, 257, 922, 5891, 1576, 438, 568, 340, 373, 645, 1049, 5975, 284, 502, 284, 3285, 326, 11, 287, 262, 6001, 286, 465, 13476, 11, 339, 550, 5710, 465, 12036, 11, 6405, 257, 5527, 27075, 11, 290, 4920, 2241, 287, 257, 4489, 64, 319, 262, 34686, 41976, 13, 357, 10915, 314, 2138, 1807, 340, 561, 423, 587, 10598, 393, 28537, 2014, 198, 198, 1, 464, 6001, 286, 465, 13476, 1, 438, 5562, 373, 644, 262, 1466, 1444, 340, 13, 314, 460, 3285, 9074, 13, 46606, 536]


## Create Dataset

In [36]:
context_length = 8
input = encoded_sample[:context_length]
target = encoded_sample[1:context_length+1]
print(f"input vector: {input}")
print(f"target vector:    {target}")

input vector: [40, 367, 2885, 1464, 1807, 3619, 402, 271]
target vector:    [367, 2885, 1464, 1807, 3619, 402, 271, 10899]


In [37]:
import torch
from torch.utils.data import Dataset, DataLoader

# create a subclass of Dataset
class GPT2DatasetV1(Dataset):
  def __init__(self,
               text,
               tokenizer,
               context_length, # length of each input vector
               stride # chunk the text into overlapping sequence of context_length
               ):
    self.input_id_vectors = []
    self.target_id_vectors = []

    # tokenize the entire text
    token_list = tokenizer.encode(text)

    # append input and target vectors
    for i in range(0, len(token_list) - context_length, stride):
      input_vector = token_list[i:i+context_length]
      target_vector = token_list[i+1:i+context_length+1]
      self.input_id_vectors.append(torch.tensor(input_vector))
      self.target_id_vectors.append(torch.tensor(target_vector))

  # get the number of input vectors
  def __len__(self):
    return len(self.input_id_vectors)

  # return the (input vector, target vector) pair
  def __getitem__(self, id):
    return self.input_id_vectors[id], self.target_id_vectors[id]

In [38]:
CONTEXT_LENGTH = 4
STRIDE = 1



dataset = GPT2DatasetV1(text=raw_text,
                        tokenizer=bpe_tokenizer,
                        context_length=CONTEXT_LENGTH,
                        stride=STRIDE)
print("number of elements: ", dataset.__len__())
print("some random pair: ", dataset.__getitem__(5))


print("-" * 60)
print("-" * 60)
print("-" * 60)
print("***EXAMPLES WITH TOKEN IDS:***")
for i in range(5):
    input_ids, target_ids = dataset[i]
    print(f"Example {i+1}")
    print("Input IDs :", input_ids.tolist())
    print("Target IDs:", target_ids.tolist())
    print("-" * 60)



print("-" * 60)
print("-" * 60)
print("-" * 60)
print("***EXAMPLES WITH TEXTS:***")
for i in range(5):
    input_ids, target_ids = dataset[i]
    input_text = bpe_tokenizer.decode(input_ids.tolist())
    target_text = bpe_tokenizer.decode(target_ids.tolist())

    print(f"Example {i+1}")
    print("Input Text :", input_text)
    print("Target Text:", target_text)
    print("-" * 60)



number of elements:  5141
some random pair:  (tensor([ 3619,   402,   271, 10899]), tensor([  402,   271, 10899,  2138]))
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
***EXAMPLES WITH TOKEN IDS:***
Example 1
Input IDs : [40, 367, 2885, 1464]
Target IDs: [367, 2885, 1464, 1807]
------------------------------------------------------------
Example 2
Input IDs : [367, 2885, 1464, 1807]
Target IDs: [2885, 1464, 1807, 3619]
------------------------------------------------------------
Example 3
Input IDs : [2885, 1464, 1807, 3619]
Target IDs: [1464, 1807, 3619, 402]
------------------------------------------------------------
Example 4
Input IDs : [1464, 1807, 3619, 402]
Target IDs: [1807, 3619, 402, 271]
------------------------------------------------------------
Example 5
Input IDs : [1807, 3619, 402, 271]
Target IDs: [3619, 402, 271, 10899]
------------

## Create a DataLoader Function That Loads the Dataset

In [39]:
def create_dataloader_V1(text,
                 batch_size=4,
                 context_length=256,
                 stride=128,
                 shuffle=True, # shuffle dataset
                 drop_last=True, # drop last batch if it not equal required size
                 num_workers=0 # number of CPU processes for preprocessing
                 ):

  tokenizer = tiktoken.get_encoding("gpt2")

  dataset = GPT2DatasetV1(text=text,
                          tokenizer=tokenizer,
                          context_length=context_length,
                          stride=stride)

  dataloader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=shuffle,
                          drop_last=drop_last,
                          num_workers=num_workers)

  return dataloader

Now let's create a dataloader instance:

In [40]:
context_length = 4

dataloader = create_dataloader_V1(text=raw_text,
                                  batch_size=8,
                                  context_length=4,
                                  stride=4,
                                  shuffle=False)

print(len(dataloader))

160


In [41]:
for i, (input_ids, target_ids) in enumerate(dataloader):
    print(f"Batch {i+1}")
    print("Input IDs :", input_ids)
    print("Target IDs:", target_ids)
    print("-" * 60)
    if i == 2:  # stop after 3 batches (0, 1, 2)
        break

Batch 1
Input IDs : tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Target IDs: tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
------------------------------------------------------------
Batch 2
Input IDs : tensor([[  287,   262,  6001,   286],
        [  465, 13476,    11,   339],
        [  550,  5710,   465, 12036],
        [   11,  6405,   257,  5527],
        [27075,    11,   290,  4920],
        [ 2241,   287,   257,  4489],
        [   64,   319,   262, 34686],
        [41

# Token Embeddings and (Absolute) Positional Embeddings

In [42]:
VOCAB_SIZE = bpe_tokenizer.n_vocab
EMBEDDING_DIM = 256

# token embeddings
token_embedding_layer = torch.nn.Embedding(num_embeddings=VOCAB_SIZE,
                                           embedding_dim=EMBEDDING_DIM)

print(token_embedding_layer)

Embedding(50257, 256)


In [43]:
input_ids

tensor([[  314,  2138,  1807,   340],
        [  561,   423,   587, 10598],
        [  393, 28537,  2014,   198],
        [  198,     1,   464,  6001],
        [  286,   465, 13476,     1],
        [  438,  5562,   373,   644],
        [  262,  1466,  1444,   340],
        [   13,   314,   460,  3285]])

In [44]:
token_embeddings = token_embedding_layer(input_ids)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [45]:
# absolute positional embedding
pos_embedding_layer = torch.nn.Embedding(num_embeddings=context_length,
                                         embedding_dim=EMBEDDING_DIM)
pos_embedding = pos_embedding_layer(torch.arange(context_length))
print(pos_embedding.shape)

torch.Size([4, 256])


In [46]:
# input embedding
input_embeddings = token_embeddings + pos_embedding
print(input_embeddings.shape)

torch.Size([8, 4, 256])


# Summary

In [47]:
# import and print out text
import urllib.request

url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/"
        "refs/heads/main/ch02/01_main-chapter-code/"
        "the-verdict.txt")
file_path = "../datasets/the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

with open(file_path, "r", encoding="utf-8") as f:
  raw_text = f.read()

print("total number of characters:", len(raw_text))
print(raw_text[:50])

total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap g


In [48]:
# create vocab
preprocessed_text = re.split(r'([,.:;"()?_!\'\s]|--)', raw_text)
preprocessed_text = [word for word in preprocessed_text if word.strip()]

# adding special tokens
list_of_all_words = sorted(list(set(preprocessed)))
list_of_all_words.extend(["<|unk|>", "<|endoftext|>"])

vocab = {token:integer for integer, token in enumerate(list_of_all_words)}

print(len(vocab.items()))

for i, word in enumerate(list(vocab.items())[-10:]):
  print(word)

1132
('year', 1122)
('years', 1123)
('yellow', 1124)
('yet', 1125)
('you', 1126)
('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|unk|>', 1130)
('<|endoftext|>', 1131)


In [49]:
# create the tokenizer class
class SimpleTokenizerV2:
  def __init__(self, vocab):
    self.word_to_id = vocab # a dict
    self.id_to_word = {id:word for word, id in vocab.items()}

  # word --> id (tokenizer)
  def encode(self, text):
    splitted_text = re.split(r'([,.:;"()?_!\'\s]|--)', text)
    word_list = [word for word in splitted_text if word.strip()]
    # handle unknown words
    word_list = [word if word in self.word_to_id
                 else "<|unk|>" for word in word_list]
    id_list = [self.word_to_id[word] for word in word_list]
    return id_list

  # id --> word (detokenizer)
  def decode(self, id_list):
    word_list = [self.id_to_word[id] for id in id_list]
    text_with_white_space = " ".join(word_list)
    text = re.sub(r'\s+([,.:;"()?_!\'\s])', r'\1', text_with_white_space)
    return text

In [50]:
text1 = """"In the midst of winter, I found, within me,
 an invincible summer" said Albert Camus"""
text2 = """Happy Hunger Games, and may the odds be ever in your favor."""
text = " <|endoftext|> ".join((text1, text2))

print("-" * 60)
print(text)
tokenizerv2 = SimpleTokenizerV2(vocab)
print("-" * 60)
print(tokenizerv2.encode(text))
print("-" * 60)
print(tokenizerv2.decode(tokenizerv2.encode(text)))

------------------------------------------------------------
"In the midst of winter, I found, within me,
 an invincible summer" said Albert Camus <|endoftext|> Happy Hunger Games, and may the odds be ever in your favor.
------------------------------------------------------------
[1, 55, 988, 1130, 722, 1130, 5, 53, 469, 5, 1130, 663, 5, 156, 1130, 1130, 1, 851, 1130, 1130, 1131, 1130, 1130, 1130, 5, 157, 662, 988, 1130, 198, 401, 568, 1128, 1130, 7]
------------------------------------------------------------
" In the <|unk|> of <|unk|>, I found, <|unk|> me, an <|unk|> <|unk|>" said <|unk|> <|unk|> <|endoftext|> <|unk|> <|unk|> <|unk|>, and may the <|unk|> be ever in your <|unk|>.


In [51]:
# use BPE
import tiktoken
bpe_tokenizer = tiktoken.get_encoding("gpt2")
encoded_text = bpe_tokenizer.encode(raw_text)

In [52]:
# create the Dataset class
import torch
from torch.utils.data import Dataset, DataLoader

class GPT2DatasetV1(Dataset):
  def __init__(self,
               text,
               tokenizer,
               context_length,
               stride):
    self.input_id_vectors_list = []
    self.target_id_vectors_list = []

    # tokenize the text
    token_list = tokenizer.encode(text)

    # append input and target vectors
    for i in range(0, len(token_list) - context_length, stride):
      input_id_vector = token_list[i:i+context_length]
      target_id_vector = token_list[i+1:i+context_length+1]
      self.input_id_vectors_list.append(torch.tensor(input_id_vector))
      self.target_id_vectors_list.append(torch.tensor(target_id_vector))

  # get the number of input vectors
  def __len__(self):
    return len(self.input_id_vectors_list)

  # return the (input vector, target vector) pair
  def __getitem__(self, id):
    return self.input_id_vectors_list[id], self.target_id_vectors_list[id]

In [53]:
# create the DataLoader function
def create_dataLoader_V1(text,
                         batch_size=4,
                         context_length=256,
                         stride=128,
                         shuffle=True,
                         drop_last=True,
                         num_workers=0):

  tokenizer = tiktoken.get_encoding("gpt2")

  dataset = GPT2DatasetV1(text=text,
                          tokenizer=tokenizer,
                          context_length=context_length,
                          stride=stride)

  dataloader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=shuffle,
                          drop_last=drop_last,
                          num_workers=num_workers)

  return dataloader

In [54]:
dataloader = create_dataloader_V1(text=raw_text,
                                  batch_size=8,
                                  context_length=4,
                                  stride=4,
                                  shuffle=False)

In [55]:
input_ids = torch.tensor([[  314,  2138,  1807,   340],
        [  561,   423,   587, 10598],
        [  393, 28537,  2014,   198],
        [  198,     1,   464,  6001],
        [  286,   465, 13476,     1],
        [  438,  5562,   373,   644],
        [  262,  1466,  1444,   340],
        [   13,   314,   460,  3285]])

In [56]:
# token embedding
VOCAB_SIZE = bpe_tokenizer.n_vocab
EMBEDDING_DIM =  256

# token embedding
token_embedding_layer = torch.nn.Embedding(num_embeddings=VOCAB_SIZE,
                                           embedding_dim=EMBEDDING_DIM)
token_embeddings = token_embedding_layer(input_ids)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [57]:
# absolute positional embedding
pos_embedding_layer = torch.nn.Embedding(num_embeddings=context_length,
                                         embedding_dim=EMBEDDING_DIM)
pos_embedding = pos_embedding_layer(torch.arange(context_length))
print(pos_embedding.shape)

torch.Size([4, 256])


In [58]:
# input embedding
input_embeddings = token_embeddings + pos_embedding
print(input_embeddings.shape)

torch.Size([8, 4, 256])


In [59]:
notebook_end_time = time.time()
runtime_in_seconds = notebook_end_time - notebook_start_time

# format as minutes and seconds
minutes, seconds = divmod(runtime_in_seconds, 60)
print(f"Notebook runtime: {int(minutes)} min {seconds:.2f} sec")

Notebook runtime: 0 min 22.37 sec
